!pip install editdistance

In [1]:
%load_ext autoreload
%autoreload 2

import os
import datetime as dt

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import models
import encoders
import decoders
from training import train, test, checkpoint, main_wandb, main_simple
from dataloaders import load_data, make_loaders, append_SOS


SEED = 27

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

wandb: Currently logged in as: raraz15 (use `wandb login --relogin` to force relogin)


cuda


In [2]:
M = 8 

data_params = {'dataset_path': '/scratch/users/udemir15/ELEC491/bassline_transcription/data/datasets/[28, 51]',
               'dataset_name': 'TechHouse_bassline_representations',
               'scale_type': 'min',
               'M': M}

X, titles = load_data(data_params)
X = append_SOS(X)

K = X.max()+1 # Number of classes, assumes consecutive [0,max] inclusive
sequence_length = X.shape[1]

print('Number of classes: {}\nSequence Length: {}'.format(K, sequence_length))
print('Number of data points: {}'.format(X.shape[0]))

Number of classes: 27
Sequence Length: 65
Number of data points: 4421


X = X[:8]

frequencies = torch.tensor(np.unique(X, return_counts=True)).to(device)
cross_entropy_weights = frequencies[1].max() / frequencies[1]
cross_entropy_weights /= cross_entropy_weights.sum()
print('Cross entropy weights:\n{}\n'.format(cross_entropy_weights))
cross_entropy_weights = torch.FloatTensor(1 / (frequencies[1] / frequencies[1].max())).to(device)
print(cross_entropy_weights)

In [ ]:
train_params = {'batch_size': 32,
               'N_epochs': 500,
               'teacher_forcing_ratio': 0.50,
               'lr': 1e-3}

encoder_params = {'input_size': K,
                  'embedding_size': 12,
                  'hidden_size': 64,
                  'n_layers': 1}

decoder_params = {'output_size': K,
                  'embedding_size': encoder_params['embedding_size'],
                  'hidden_size': encoder_params['hidden_size'],
                  'n_layers': encoder_params['n_layers']}

params = {'data_params': data_params,
          'encoder_params': encoder_params,
          'decoder_params':decoder_params, 
          'train_params':train_params}

encoder = encoders.Encoder(**encoder_params)
decoder = decoders.Decoder(**decoder_params)

model = models.Seq2Seq(encoder, decoder, device, train_params['teacher_forcing_ratio']).to(device)

print(model)
print('Number of parameters: {}'.format(sum([parameter.numel() for parameter in model.parameters()])))

criterion = nn.CrossEntropyLoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=train_params['lr'])

train_loader, validation_loader, test_loader = make_loaders(X, train_params)

main_wandb(model, criterion, optimizer, device, train_loader, validation_loader, test_loader, params)
#main_simple(model, criterion, optimizer, device, train_loader, validation_loader, test_loader, params)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(27, 12)
    (rnn): LSTM(12, 64, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(27, 12)
    (rnn): LSTM(12, 64, batch_first=True)
    (fc_out): Linear(in_features=64, out_features=27, bias=True)
  )
)
Number of parameters: 42339


  0%|          | 0/500 [00:00<?, ?it/s]


Before Training:
Test Loss: 3.2952, Test Accuracy: 0.0332, Test Distance: 60.77

Sample:
tensor([25, 10, 10, 10, 20, 20,  8, 13, 13, 13, 13, 13, 13,  1, 20, 16, 18,  4,
        12, 16, 24, 16, 24, 16, 24, 16, 20, 16, 24, 16, 24, 16, 20, 16, 24, 16,
        24, 16, 20, 16, 24, 16, 24, 16, 20, 16, 24, 16, 24, 16, 20, 16, 24, 16,
        24, 16, 20, 16, 24, 16, 24, 16, 20, 16], device='cuda:0')



  0%|          | 1/500 [00:07<1:00:58,  7.33s/it]

Epoch: 0, train_loss: 2.295801, train_acc: 0.399, val_loss: 2.032781, val_acc: 2.033
Sample:
tensor([16, 22,  0,  2,  1, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26,
        26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26,
        26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26,
        26, 26, 26, 26, 26, 26, 26, 26, 26, 26], device='cuda:0')


  2%|▏         | 9/500 [01:02<56:48,  6.94s/it]  

In [ ]:
test_loss, test_acc = test(model, test_loader, criterion, device)
samples = model.sample()

for epoch in tqdm(range(train_params['N_epochs'])):

    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    validation_loss, validation_acc = test(model, validation_loader, criterion, device)
    
    model.decoder.update_teacher_forcing_ratio(decoder_params['teacher_forcing_ratio']/train_params['N_epochs'])
    
    if epoch+1 % 5:
        print('Epoch: {}, train_loss: {:.6f}, acc: {:.3f}'.format(epoch+1, train_loss, train_acc))

test_loss, test_acc = test(model, test_loader, criterion, device)
print('Test Loss After Training: {:.6f}'.format(test_loss))